In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ==========================================================
# 🚀 Heart Disease - Kaggle Playground 2026 (Stable Version)
# ==========================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# =========================
# 📥 Load Data
# =========================
train = pd.read_csv("/kaggle/input/playground-series-s6e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")

# =========================
# 🎯 Target + ID
# =========================
target_col = "Heart Disease"

y = train[target_col]
test_ids = test["id"]

X = train.drop(columns=[target_col, "id"])
X_test = test.drop(columns=["id"])

# =========================
# 🔁 Stratified KFold (3-fold)
# =========================
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):

    print(f"\n===== Fold {fold+1} =====")

    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        iterations=1500,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        eval_metric="AUC",
        random_seed=42,
        early_stopping_rounds=150,
        verbose=200
    )

    model.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val)
    )

    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / skf.n_splits

    fold_auc = roc_auc_score(y_val, oof_preds[val_idx])
    print(f"Fold {fold+1} AUC: {fold_auc:.6f}")

# =========================
# 📊 Final CV Score
# =========================
overall_auc = roc_auc_score(y, oof_preds)
print("\nOverall CV AUC:", overall_auc)

# =========================
# 📝 Submission
# =========================
submission = pd.DataFrame({
    "id": test_ids,
    "Heart Disease": test_preds
})

submission.to_csv("submission.csv", index=False)

print("\n✅ submission.csv created successfully")
